In [14]:
# imports
import heapq
import re

import numpy as np
import pandas as pd

In [2]:
# inputs
test_data = open("input-test.txt").read()
data = open("input.txt").read()

# Part 1

In [7]:
# test
content = test_data


def parse_input(content):
    array = np.array(list(map(list, content.split("\n"))), dtype=int)
    return array


# https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm#Algorithm
def find_optimal_path_djikstra(array, fill_value=99999):
    current = (0, 0)
    end = tuple(np.array(array.shape) - [1, 1])
    cost_from_zero = np.full_like(array, fill_value=fill_value, dtype=int)
    cost_from_zero[current] = 0
    unvisited = {(x, y) for x in range(array.shape[1]) for y in range(array.shape[0])}

    while True:
        neighbors = set((current[0] + x, current[1] + y) for x, y in [[-1, 0], [1, 0], [0, -1], [0, 1]])
        neighbors &= unvisited

        ccost = cost_from_zero[current]
        for nb in neighbors:
            if ccost + array[nb] < cost_from_zero[nb]:
                cost_from_zero[nb] = ccost + array[nb]

        unvisited -= {current}
        init_cands = {node for node in unvisited if cost_from_zero[node] == fill_value}
        if current == end or len(unvisited) == len(init_cands):
            break

        current = sorted(unvisited - init_cands, key=lambda x: cost_from_zero[x])[0]

    return cost_from_zero[end]


array = parse_input(content)
assert find_optimal_path_djikstra(array) == 40

In [42]:
# from https://bradfieldcs.com/algos/graphs/dijkstras-algorithm/


def create_graph(array):
    return {
        f"a_{x}_{y}": {
            f"a_{x+i}_{y+j}": array[y + j][x + i]
            for i, j in [[-1, 0], [1, 0], [0, -1], [0, 1]]
            if x + i >= 0 and y + j >= 0 and x + i < array.shape[1] and y + j < array.shape[0]
        }
        for x in range(array.shape[1])
        for y in range(array.shape[0])
    }


def calculate_distances(graph, starting_vertex):
    distances = {vertex: float("infinity") for vertex in graph}
    distances[starting_vertex] = 0

    pq = [(0, starting_vertex)]
    while len(pq) > 0:
        current_distance, current_vertex = heapq.heappop(pq)

        # Nodes can get added to the priority queue multiple times. We only
        # process a vertex the first time we remove it from the priority queue.
        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            # Only consider this new path if it's better than any path we've
            # already found.
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(pq, (distance, neighbor))

    return distances

In [43]:
# test
content = test_data
array = parse_input(content)
graph = create_graph(array)
end_node = f"a_{array.shape[1]-1}_{array.shape[0]-1}"
assert calculate_distances(graph, "a_0_0")[end_node] == 40

In [44]:
# real
content = data
array = parse_input(content)
graph = create_graph(array)
end_node = f"a_{array.shape[1]-1}_{array.shape[0]-1}"
calculate_distances(graph, "a_0_0")[end_node]

702

# Part 2

In [45]:
# test
content = test_data


def parse_input(content, extend=(1, 1)):
    array = np.array(list(map(list, content.split("\n"))), dtype=int)
    multi_array = [[(array + i + j - 1) % 9 + 1 for i in range(extend[0])] for j in range(extend[1])]
    return np.block(multi_array)


array = parse_input(content, extend=(5, 5))
graph = create_graph(array)
end_node = f"a_{array.shape[1]-1}_{array.shape[0]-1}"
assert calculate_distances(graph, "a_0_0")[end_node] == 315

In [47]:
# real
content = data
array = parse_input(content, extend=(5, 5))
graph = create_graph(array)
end_node = f"a_{array.shape[1]-1}_{array.shape[0]-1}"
calculate_distances(graph, "a_0_0")[end_node]

2955